In [54]:
import numpy as np
import pandas as pd
import acquire as a
import requests
from bs4 import BeautifulSoup
import re

In [2]:
data = a.scrape_github_data()

#### By adding repo names to the REPOS area of the acquire.py file, in the format 'gocodeup/codeup-setup-script' (no spaces) they will be added to the list of repo readme contents that will be scraped.

In [3]:
data

[{'repo': 'gocodeup/codeup-setup-script',
  'language': 'Shell',
  'readme_contents': '# Codeup Setup Script\n\nSetup script for Codeup students\' laptops to install all of the tools we will\nneed for the java course. We will install the following:\n\n- [xcode](https://developer.apple.com/xcode/features/): command line tools for\n  macs\n- [brew](http://brew.sh/): package manager for macs\n- [java](https://en.wikipedia.org/wiki/Java_(programming_language))\n- [tomcat](http://tomcat.apache.org/): the java webserver\n- [maven](https://maven.apache.org/): a java dependency and build management tool\n- [mysql](https://www.mysql.com/): the database we\'ll use for the class\n- [node js](https://nodejs.org/en/): a JavaScript runtime outside the browser\n- [npm](https://www.npmjs.com/): a package manager for JavaScript\n- [intellij](https://www.jetbrains.com/idea/): a Java IDE\n\nIn addition, we will:\n\n- setup ssh keys for the student\'s laptop and guide them through the process of\n  linkin

#### Our first step should be to scrape a list of repo names. On github the repo names have spaces, so the spaces will also need to be removed before they can be added to the REPOS list.

In [5]:
# nlp projects using python
python_url = 'https://github.com/topics/nlp?l=python'
# nlp projects using html
html_url = 'https://github.com/topics/nlp?l=html'

In [7]:
response = requests.get('https://github.com/topics/nlp?l=python')

In [8]:
response

<Response [200]>

In [9]:
soup = BeautifulSoup(response.content)

In [39]:
# soup

In [43]:
# grab one project repo name from the soup
soup.find_all('a', class_="text-bold wb-break-word")[0]['href']

'/huggingface/transformers'

In [47]:
len(soup.find_all('a', class_="text-bold wb-break-word"))

20

In [52]:
python_url_list = [item['href'] for item in 
                   soup.find_all('a', class_="text-bold wb-break-word")]

In [53]:
python_url_list

['/huggingface/transformers',
 '/apachecn/ailearning',
 '/google-research/bert',
 '/hankcs/HanLP',
 '/explosion/spaCy',
 '/RasaHQ/rasa',
 '/huggingface/datasets',
 '/mindsdb/mindsdb',
 '/RaRe-Technologies/gensim',
 '/flairNLP/flair',
 '/microsoft/unilm',
 '/nltk/nltk',
 '/PaddlePaddle/PaddleHub',
 '/allenai/allennlp',
 '/chiphuyen/stanford-tensorflow-tutorials',
 '/PaddlePaddle/PaddleNLP',
 '/deepset-ai/haystack',
 '/sloria/TextBlob',
 '/ymcui/Chinese-BERT-wwm',
 '/ymcui/Chinese-LLaMA-Alpaca']

#### There is 20, now to figure out how to get more results

<form class="ajax-pagination-form js-ajax-pagination" data-turbo="false" action="https://github.com/topics/nlp?l=python" accept-charset="UTF-8" method="get">
    <input name="page" type="hidden" value="2">
    <button type="submit" class="ajax-pagination-btn btn btn-outline color-border-default f6 mt-0 width-full" data-disable-with="Loading more…">
      Load more…
    </button>
</form>
<input name="page" type="hidden" value="2">